In [1]:
!pip install langchain 

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [4]:
from langchain import HuggingFaceHub

In [47]:
# llm_huggingface=HuggingFaceHub(repo_id="Falconsai/text_summarization")
llm=HuggingFaceHub(repo_id="google/flan-t5-base", model_kwargs={"temperature":0,"max_length":64})

In [48]:
llm.invoke('can you tell me about capital India?')

'Delhi'

### Prompt Templates & LLMChain

In [49]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate(input_variables=['Country'], template="Tell me the capital of this {country}")

In [50]:
prompt_template.format(country="India")

'Tell me the capital of this India'

In [51]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm,prompt=prompt_template)
# chain.invoke('India')
print(chain.run('India'))

Delhi


### Combining Multiple Chains using Simple Squential Chains

In [52]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_template)

famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")

In [53]:
famous_chain=LLMChain(llm=llm,prompt=famous_template)

In [54]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains=[capital_chain,famous_chain],
)

In [55]:
chain.run('India')

"Delhi's most famous landmarks"

### Summarize Model: facebook/bart-large-cnn

In [38]:
# facebook/bart-large-cnn
llm_summary=HuggingFaceHub(repo_id="facebook/bart-large-cnn")
summary_template = PromptTemplate(input_variables=['Summarize'], 
                                  template="please summarize this article {summarize}")
chain=LLMChain(llm=llm_summary,prompt=summary_template)
article = """The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."""
chain.run(article)

'The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world.'

### Sequential Chain

In [56]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_template,output_key="capital")

In [57]:
famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")

famous_chain=LLMChain(llm=llm,prompt=famous_template,output_key="places")

In [58]:
from langchain.chains import SequentialChain
chain=SequentialChain(chains=[capital_chain,famous_chain],
input_variables=['country'],
output_variables=['capital',"places"])

In [59]:
chain({'country':"India"})

/Users/ajay/Documents/Personal/projects/AGI/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'country': 'India',
 'capital': 'Delhi',
 'places': "Delhi's most famous landmarks"}